# Evaluate Accuracy and Loss on Test Set

## Preliminaries: iPython magic and imports

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import h5py
from pprint import pprint
import copy

## Read in the predictions and make plots

In [3]:
# Path to the directory where all data is stored
data_path = '../data'

# Name of the file containing the predictions we want to inspect
file_name = 'predictions_GW170104_0700_1200_8k.h5'
file_path = os.path.join(data_path, 'predictions', 'timeseries', file_name)

# Open the HDF file and read in the spectrogram, predictions and true labels
with h5py.File(file_path, 'r') as file:
    timeseries  = np.array(file['x'])
    pred_labels = np.array(file['y_pred'])
    true_labels = np.array(file['y_true'])

    loss = float(np.array(file['avg_loss']))
    hamming = float(np.array(file['avg_hamming_distance']))

print('FROM COMPUTATION DURING TRAINING:')
print('Loss: {:.3f}'.format(loss))
print('Hamming Distance: {:.3f}'.format(hamming), end='\n\n')

FROM COMPUTATION DURING TRAINING:
Loss: 0.189
Hamming Distance: 0.062



In [4]:
print('CALCULATED HERE:')

pred_labels = pred_labels
true_labels = true_labels

for threshold in [0.5]:
    
    def apply_threshold(x):
        # threshold = 0.9
        return int(x >= threshold)
    apply_threshold = np.vectorize(apply_threshold)

    window_size = 1
    smoothed_pred_labels = np.empty(pred_labels.shape)
    for i in range(len(pred_labels)):
        smoothed_pred_labels[i] = np.convolve(pred_labels[i], np.ones(window_size)/window_size, mode='same')
    
    thresh_pred_labels = apply_threshold(smoothed_pred_labels)
    accuracy = 1 - np.mean(np.abs(np.ceil(true_labels) - thresh_pred_labels), axis=(1, 0))

    print('Accurary @ {:.2f}: {:.3f}'.format(threshold, accuracy))

CALCULATED HERE:
Accurary @ 0.50: 0.938
